In [0]:
!pip install pyspark

     |▍   

*** WARNING: max output size exceeded, skipping output. ***

     |████████████████████████████████| 317.3 MB 9.0 kB/s 
     |████████████████████████████████| 200 kB 89.8 MB/s 
  Created wheel for pyspark: filename=pyspark-3.5.3-py2.py3-none-any.whl size=317840651 sha256=0902d5a98880bcf69d6991003fde2b3938984e2eb10380cd569b9ed403fda786
  Stored in directory: /root/.cache/pip/wheels/2e/d2/18/6f4f20e8332359f7fffceb6828edcc80ef96f86744192a7bb9
Successfully built pyspark
You should consider upgrading via the '/local_disk0/.ephemeral_nfs/envs/pythonEnv-d3d6a547-07a6-4c67-a23e-bcc834c32416/bin/python -m pip install --upgrade pip' command.


In [0]:
from pyspark.sql import SparkSession
from pyspark.sql.types import StructType,StructField,IntegerType,StringType
import pyspark.sql.functions as f
from pyspark.sql.window import Window

Create a SparkSession

In [0]:
spark=SparkSession.builder.appName("BadmintonAnalyis").getOrCreate()

Create Schema

In [0]:
schema=StructType([StructField("user_id",IntegerType(),True),StructField("kit_id",IntegerType(),True),StructField("login_date",StringType(),True),StructField("sessions_count",IntegerType(),True)])

In [0]:
Data=[(1,2,"2016-03-01",5),(1,2,"2016-03-02",6),(2,3,"2017-06-25",1),(3,1,"2016-03-02",0),(3,4,"2018-07-03",5)]

Create Dataframe


In [0]:
df=spark.createDataFrame(Data,schema=schema)

In [0]:
df.show()

+-------+------+----------+--------------+
|user_id|kit_id|login_date|sessions_count|
+-------+------+----------+--------------+
|      1|     2|2016-03-01|             5|
|      1|     2|2016-03-02|             6|
|      2|     3|2017-06-25|             1|
|      3|     1|2016-03-02|             0|
|      3|     4|2018-07-03|             5|
+-------+------+----------+--------------+



First Approach Use GroupBy and Min

In [0]:
df1=df.groupBy("user_id").agg(f.min("login_date").alias("min_date"))

In [0]:
df1.show()

+-------+----------+
|user_id|  min_date|
+-------+----------+
|      1|2016-03-01|
|      2|2017-06-25|
|      3|2016-03-02|
+-------+----------+



Secong Approach Using window functions

In [0]:
df.show()

+-------+------+----------+--------------+
|user_id|kit_id|login_date|sessions_count|
+-------+------+----------+--------------+
|      1|     2|2016-03-01|             5|
|      1|     2|2016-03-02|             6|
|      2|     3|2017-06-25|             1|
|      3|     1|2016-03-02|             0|
|      3|     4|2018-07-03|             5|
+-------+------+----------+--------------+



In [0]:
Division=Window.partitionBy("user_id").orderBy("login_date")

In [0]:
df2=df.withColumn("ran",f.rank().over(Division))

In [0]:
df2.filter(df2.ran==1).select("user_id","login_date").show()

+-------+----------+
|user_id|login_date|
+-------+----------+
|      1|2016-03-01|
|      2|2017-06-25|
|      3|2016-03-02|
+-------+----------+



Total Matches Played

In [0]:
df.withColumn("total_login_tilldate",f.sum(f.col("sessions_count")).over(Division)).show()

+-------+------+----------+--------------+--------------------+
|user_id|kit_id|login_date|sessions_count|total_login_tilldate|
+-------+------+----------+--------------+--------------------+
|      1|     2|2016-03-01|             5|                   5|
|      1|     2|2016-03-02|             6|                  11|
|      2|     3|2017-06-25|             1|                   1|
|      3|     1|2016-03-02|             0|                   0|
|      3|     4|2018-07-03|             5|                   5|
+-------+------+----------+--------------+--------------------+



Users Who came for atlease 2 consecutive days after their first login


In [0]:
df3=df2.withColumn("nextdate",f.lead(f.col("login_date")).over(Division))

In [0]:
df3.filter(f.col("ran")==1).withColumn("differencedate",f.datediff(f.col("nextdate"),f.col("login_date"))).filter(f.col("differencedate")==1).select("user_id").show()

+-------+
|user_id|
+-------+
|      1|
+-------+

